In [ ]:
from pathlib import Path
import torch


from unet.unet_model import UNet
from unet.unet_dataset.image_and_masks.image_masks import ImageMasksDataset
from unet.train_unet_utils import collate_fn, train

from unet.train_unet_utils import unbatch, train_batch, validate_batch

In [ ]:
configs = {
    "device": "cpu",
    "in_channels": 3,
    "out_channels": 1,
    "conv_channels": [16, 32, 64],
    "data_dir": "../data/water_bodies_segmentation/",
    "train_image_folder": "train_images",
    "train_mask_folder": "train_masks",
    "test_image_folder": "test_images",
    "test_mask_folder": "test_masks",
    "batch_size": 4,
    "epochs": 100,
}

In [ ]:
# Device.
device = configs.get(
    "device", 
    torch.device("cuda" if torch.cuda.is_available() else "cpu")
)
if torch.cuda.is_available() is False and device == torch.device("cuda"):
    device = torch.device("cpu")
print("Using device: {}.".format(device))

In [ ]:
# U-Net.
in_channels = configs.get("in_channels", 3)
out_channels = configs.get("out_channels", 1)
conv_channels = configs.get("conv_channels", [64, 128, 256, 512, 1024])

model = UNet(in_channels, out_channels, conv_channels)
model.to(device)
model.train()

print("U-Net in_channels={}, out_channels={}, conv_channels={}.".format(
    in_channels, out_channels, conv_channels)
)

In [ ]:
# Dataloaders.
data_dir = configs.get("data_dir", None)
train_image_folder = configs.get("train_image_folder", "train_images")
train_mask_folder = configs.get("train_mask_folder", "train_masks")
test_image_folder = configs.get("test_image_folder", "test_images")
test_mask_folder = configs.get("test_mask_folder", "test_masks")

data_dir = Path(data_dir)

# Train dataset. 
train_ds = ImageMasksDataset(
    data_dir=data_dir, image_folder=train_image_folder, mask_folder=train_mask_folder, train=True
)
# Test dataset.
test_ds = ImageMasksDataset(
    data_dir=data_dir, image_folder=test_image_folder, mask_folder=test_mask_folder, train=False
)

print(len(train_ds), len(test_ds))

In [ ]:
# Create the DataLoaders from the Datasets. 
batch_size = configs.get("batch_size", 4)
n_epochs = configs.get("epochs", 100)

train_dl = torch.utils.data.DataLoader(
    train_ds, batch_size = batch_size, shuffle = False, collate_fn = collate_fn,
)

test_dl = torch.utils.data.DataLoader(
    test_ds, batch_size = batch_size, shuffle = False, collate_fn = collate_fn,
)

print(len(train_dl), len(test_dl))

In [ ]:
# Set up optimizer and loss function.
loss_function = torch.nn.BCELoss()

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr = 0.005, momentum = 0.9, weight_decay = 0.0005)

print(loss_function) 
print(optimizer)

In [ ]:
# Train the model.
model, train_losses, test_losses = train(model, optimizer, loss_function, n_epochs, train_dl, test_dl, device, True)